## Preambule

In [1]:
# General packages
import json
from pathlib import Path

# Plotting
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr
from plotly.colors import n_colors
from plotly.subplots import make_subplots
from tqdm import tqdm

## Parameters

In [2]:
rules = ["PCC", "ECPC", "AP"]
rulecolors = ["goldenrod", "tomato", "forestgreen"]

## Paths

In [3]:
path_main = Path("K:/Code/effort-sharing/")
path_data = Path("K:/Data/Data_effortsharing/DataUpdate_ongoing/")
path_figs = path_main / "Figures" / "Paper_FairShares"

## Read data files

In [4]:
xr_dataread = xr.open_dataset(path_data / "startyear_2021/xr_dataread.nc")
xr_traj_2030 = xr.open_dataset(
    path_data / "startyear_2021/Aggregated_files/xr_alloc_2030_GHG_incl.nc"
)
xr_traj_2040 = xr.open_dataset(
    path_data / "startyear_2021/Aggregated_files/xr_alloc_2040_GHG_incl.nc"
)
xr_total = xr.open_dataset(path_data / "xr_policyscen.nc")

all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

## Get Cost-optimal output

In [5]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(["Emissions|Kyoto Gases", "Policy Cost|Consumption Loss"])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel(
    "X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx",
    sheet_name="meta_Ch3vetted_withclimate",
)
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6_meta["ModelScenario"] = modscens_meta
df_ar6_meta = df_ar6_meta[["ModelScenario", "Category", "Policy_category"]]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(["C1"])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6["ModelScenario"] = modscens
df_ar6 = df_ar6.drop(["Model", "Scenario", "Unit"], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(
    id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time=np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

In [6]:
cost_opt = (
    xr_scen_r.sel(Variable="Emissions|Kyoto Gases", Time=2040)
    / xr_scen_r.sel(Variable="Emissions|Kyoto Gases", Time=2040)
    * xr_scen_r.sel(Variable="Emissions|Kyoto Gases", Time=2040)
)

## Compute fractions

In [7]:
xr_larger_than_costopt = (
    xr_traj_2040.sel(
        Temperature=1.5,
        Convergence_year=2050,
        Risk=0.5,
        NegEmis=0.5,
        NonCO2red=0.5,
        Timing="Immediate",
    ).median(dim=["Scenario"])
    > cost_opt.mean(dim="ModelScenario").Value
)

World

In [8]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_World_v1.1.csv")
df_ar6 = df_ar6[
    df_ar6.Variable.isin(["Emissions|Kyoto Gases", "Policy Cost|Consumption Loss", "Price|Carbon"])
]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel(
    "X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx",
    sheet_name="meta_Ch3vetted_withclimate",
)
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6_meta["ModelScenario"] = modscens_meta
df_ar6_meta = df_ar6_meta[["ModelScenario", "Category", "Policy_category"]]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(["C1"])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6["ModelScenario"] = modscens
df_ar6["Region"] = "WORLD"
df_ar6 = df_ar6.drop(["Model", "Scenario", "Unit"], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(
    id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen = xr.Dataset.from_dataframe(dummy)
xr_scen = xr_scen.reindex(Time=np.arange(1850, 2101))
xr_scen = xr_scen.interpolate_na(dim="Time", method="linear")

Regional

In [9]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[
    df_ar6.Variable.isin(["Emissions|Kyoto Gases", "Policy Cost|Consumption Loss", "Price|Carbon"])
]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel(
    "X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx",
    sheet_name="meta_Ch3vetted_withclimate",
)
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6_meta["ModelScenario"] = modscens_meta
df_ar6_meta = df_ar6_meta[["ModelScenario", "Category", "Policy_category"]]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(["C1"])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6["ModelScenario"] = modscens
df_ar6 = df_ar6.drop(["Model", "Scenario", "Unit"], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(
    id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time=np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

R-10 regions

In [10]:
df_ar6 = pd.read_csv(
    "X:/user/dekkerm/Data/IPCC/AR6_R10/AR6_Scenarios_Database_R10_regions_v1.1.csv"
)
df_ar6 = df_ar6[
    df_ar6.Variable.isin(["Emissions|Kyoto Gases", "Policy Cost|Consumption Loss", "Price|Carbon"])
]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel(
    "X:/user/dekkerm/Data/IPCC/AR6_R10/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx",
    sheet_name="meta_Ch3vetted_withclimate",
)
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6_meta["ModelScenario"] = modscens_meta
df_ar6_meta = df_ar6_meta[["ModelScenario", "Category", "Policy_category"]]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(["C1"])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6["ModelScenario"] = modscens
df_ar6 = df_ar6.drop(["Model", "Scenario", "Unit"], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(
    id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r10 = xr.Dataset.from_dataframe(dummy)
xr_scen_r10 = xr_scen_r10.reindex(Time=np.arange(1850, 2101))
xr_scen_r10 = xr_scen_r10.interpolate_na(dim="Time", method="linear")

In [11]:
df_conv = pd.read_excel("X:/user/dekkerm/Data/AR6_regionclasses.xlsx", sheet_name="Sheet1")[
    ["ISO", "region_ar6_10"]
]
regs = np.array(df_conv.region_ar6_10)
regs[regs == "Africa"] = "R10AFRICA"
regs[regs == "Eastern Asia"] = "R10CHINA+"
regs[regs == "Europe"] = "R10EUROPE"
regs[regs == "Southern Asia"] = "R10INDIA+"
regs[regs == "Latin America and Caribbean"] = "R10LATIN_AM"
regs[regs == "Middle East"] = "R10MIDDLE_EAST"
regs[regs == "North America"] = "R10NORTH_AM"
regs[regs == "Asia-Pacific Developed"] = "R10PAC_OECD"
regs[regs == "Eurasia"] = "R10REF_ECON"
regs[regs == "South-East Asia and developing Pacific"] = "R10REST_ASIA"
df_conv["region_ar6_10"] = regs

In [12]:
native_regions = np.array(xr_scen_r.Region)
xrs = []
for cty in all_countries_iso:
    if cty not in native_regions:
        which_group = np.array(df_conv[df_conv.ISO == cty].region_ar6_10)[0]
        group_members = np.array(df_conv[df_conv.region_ar6_10 == which_group].ISO)
        emis_total = float(
            xr_dataread.GHG_hist.sel(
                Region=np.intersect1d(group_members, all_countries_iso), Time=2021
            ).sum(dim="Region")
        )
        emis_frac = xr_dataread.GHG_hist.sel(Region=cty, Time=2021) / emis_total
        xrry = xr.merge(
            [
                xr_scen_r10.sel(Region=which_group, Variable=["Emissions|Kyoto Gases"]) * emis_frac,
                xr_scen_r10.sel(Region=which_group, Variable=["Price|Carbon"]),
            ]
        ).expand_dims({"Region": [cty]})
        xrs.append(xrry)
xr_scen_infilled_r = xr.concat(xrs, dim="Region")

Concatenate

In [13]:
xr_scen_all = xr.merge([xr_scen, xr_scen_r, xr_scen_infilled_r])

## Data for plots

In [14]:
settings_default = {
    "Temperature": 1.6,
    "Risk": 0.5,
    "NegEmis": 0.5,
    "NonCO2red": 0.5,
    "Timing": "Immediate",
    "Convergence_year": 2050,
    "Scenario": "SSP2",
    "Discount_factor": 0,
    "Historical_startyear": 1990,
}
#'Capability_threshold': "Th",
#'RCI_weight': "Half"}
settings_ranges = {
    "Temperature": [1.6, 2.0],
    "Risk": [0.5, 0.33],
    "Convergence_year": [2050, 2080],
    "NonCO2red": [0.33, 0.5, 0.67],
    "Timing": ["Immediate"],
}
#'Capability_threshold': "Th",
#'RCI_weight': "Half"}

In [15]:
ndcs = xr_dataread.GHG_ndc.mean(dim="Ambition").min(dim="Conditionality")
ndcs = ndcs.where(ndcs.Region != "ZMB", np.nan)
ndcs = ndcs.where(ndcs.Region != "COM", np.nan)
data = xr_traj_2030.sel(**settings_ranges)[["ECPC", "AP", "PCC"]]
data_max = xr.concat(
    [data.ECPC, data.AP.expand_dims({"Convergence_year": [2050, 2080]}), data.PCC], dim="variable2"
).max(dim=list(settings_default.keys()) + ["variable2"])
data = xr_traj_2030.sel(**settings_default)[["ECPC", "AP", "PCC"]]
data_def = xr.concat([data.ECPC, data.AP, data.PCC], dim="variable2").max(dim="variable2")

In [16]:
print("Preferences on default settings")
print("-----")
data = xr_traj_2030.sel(**settings_default)[["ECPC", "AP", "PCC"]]
data_def = xr.concat([data.ECPC, data.AP, data.PCC], dim="variable2").max(dim="variable2")
argmax_default = (
    xr.concat([data.ECPC, data.AP, data.PCC], dim="variable2").fillna(-9e99).argmax(dim="variable2")
)

data = xr_traj_2030.sel(**settings_ranges)[["ECPC", "AP", "PCC"]]
data_max = xr.concat(
    [data.ECPC, data.AP.expand_dims({"Convergence_year": [2050, 2080]}), data.PCC], dim="variable2"
).max(dim=list(settings_default.keys()) + ["variable2"])
argmax_maximum = (
    xr.concat(
        [data.ECPC, data.AP.expand_dims({"Convergence_year": [2050, 2080]}), data.PCC],
        dim="variable2",
    )
    .fillna(-9e99)
    .argmax(dim=list(settings_default.keys()) + ["variable2"])
)
x = xr.concat(
    [data.ECPC, data.AP.expand_dims({"Convergence_year": [2050, 2080]}), data.PCC], dim="variable2"
).fillna(-9e99)
argmax_max = []
maxs = x.max(dim=list(settings_default.keys()) + ["variable2"])
for cty in x.Region:
    mx = float(maxs.sel(Region=cty).values)
    wh = np.where(x.sel(Region=cty).values == mx)
    wh = np.array(wh)[:, 0]
    argmax_max.append(wh[0])
argmax_max = np.array(argmax_max)

df_c = []
df_r_d = []
df_r_m = []
for cty_i, cty in enumerate(all_countries_iso):
    df_c.append(all_countries_names[cty_i])
    df_r_d.append(["ECPC", "AP", "PCC"][argmax_default.sel(Region=cty).values])
    df_r_m.append(["ECPC", "AP", "PCC"][argmax_max[cty_i]])
df_dict = {"Region": df_c, "Default": df_r_d, "Maximum": df_r_m}
df = pd.DataFrame(df_dict)
df.to_csv(path_data / "preferences_default_max.csv")

Preferences on default settings
-----


In [17]:
fair = xr_traj_2030.sel(
    Temperature=1.6,
    Risk=0.5,
    Timing="Immediate",
    NonCO2red=0.5,
    NegEmis=0.5,
    Scenario="SSP2",
    Convergence_year=2050,
    Discount_factor=0,
    Historical_startyear=1990,
)[["PCC", "AP", "ECPC"]]
fair_all = xr_traj_2030.sel(
    Temperature=[1.6, 2.0],
    Risk=[0.5, 0.33],
    Timing=["Immediate"],
    NonCO2red=[0.33, 0.5, 0.67],
    NegEmis=[0.33, 0.5, 0.67],
    Convergence_year=[2050, 2080],
)[["PCC", "AP", "ECPC"]]

In [18]:
GAP_DEFAULT = float(
    (fair.to_array().max(dim="variable") - ndcs).sel(Region=list(all_countries_iso) + ["EU"]).sum()
)
GAP_MAX = float(
    (fair_all.to_array().max(dim=list(settings_default.keys()) + ["variable"]) - ndcs)
    .sel(Region=list(all_countries_iso) + ["EU"])
    .sum()
)

In [19]:
xr_policyscen = xr.open_dataset(path_data / "xr_policyscen.nc")
curpols = xr_policyscen.CurPol.sel(Time=2030).mean(dim="Model")
nzs = xr_policyscen.NetZero.sel(Time=2030).mean(dim="Model")

In [20]:
path_ctygroups = "X:/user/dekkerm/Data/" + "UNFCCC_Parties_Groups_noeu.xlsx"
df = pd.read_excel(path_ctygroups, sheet_name="Country groups")
countries_iso = np.array(df["Country ISO Code"])
countries_name = np.array(df["Name"])
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [21]:
costopts = xr_scen_all.sel(Time=2030, Variable="Emissions|Kyoto Gases").mean(dim="ModelScenario")

## Gaps

In [22]:
nec = xr_dataread.GHG_globe.sel(
    Time=2030, Temperature=1.6, Risk=0.5, NonCO2red=0.5, NegEmis=0.5, Timing="Immediate"
).values
gap_default = float(data_def.sel(Region=countries_iso).sum() - nec)
gap_default_perc = float((data_def.sel(Region=countries_iso).sum() - nec) / nec)

gap_max = float(data_max.sel(Region=countries_iso).sum() - nec)
gap_max_perc = float((data_max.sel(Region=countries_iso).sum() - nec) / nec)

In [23]:
xr_alloc_2030 = xr.open_dataset(
    path_data / "startyear_2021/Aggregated_files/xr_alloc_2030_GHG_incl.nc"
)

## New plot

In [25]:
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[
        [{"type": "choropleth"}, {"type": "choropleth"}],
        [{"type": "choropleth"}, {"type": "choropleth"}],
    ],
    horizontal_spacing=0.0,
    vertical_spacing=0.0,
)
for i in range(4):
    dath = [
        (ndcs - data_def) / data_def,
        (ndcs - data_max) / data_max,
        (costopts.Value) / data_def - 1,
        (costopts.Value) / data_max - 1,
    ][i]
    dat = np.array(dath)
    # if i>2:
    #     dat[dat < 0] = 0
    # scale = ['RdBu_r', 'RdBu_r', 'PuOr_r', 'PuOr_r'][i]
    scale = ["RdBu_r", "RdBu_r", "BrBG_r", "BrBG_r"][i]
    dc = [
        dict(
            orientation="h",
            x=0.5,
            y=0.5,
            xanchor="center",
            yanchor="bottom",
            title="NDC deviation from emission allocation in 2030",
            tickvals=[-1, -0.5, 0, 0.5, 1],
            ticktext=["100%<br>NDC sufficient <", "50%", "0%", "50%", "100%<br>> NDC insufficient"],
            tickfont=dict(size=14),
            len=0.5,
            lenmode="fraction",
            thickness=20,
            thicknessmode="pixels",
            titlefont=dict(size=14),
        ),
        dict(
            orientation="h",
            x=0.5,
            y=0.5,
            xanchor="center",
            yanchor="bottom",
            title="NDC deviation from emission allocation in 2030",
            tickvals=[-1, -0.5, 0, 0.5, 1],
            ticktext=["100%<br>NDC sufficient <", "50%", "0%", "50%", "100%<br>> NDC insufficient"],
            tickfont=dict(size=14),
            len=0.5,
            lenmode="fraction",
            thickness=20,
            thicknessmode="pixels",
            titlefont=dict(size=14),
        ),
        dict(
            orientation="h",
            x=0.5,
            y=0.0,
            xanchor="center",
            yanchor="bottom",
            title="Fraction of mitigation in 2030 non-domestic",
            tickvals=[-1, -0.5, 0, 0.5, 1],
            ticktext=[
                "100%<br>More allocated than cost-optimal",
                "50%",
                "0%",
                "50%",
                "100%<br>Less allocated than cost-optimal",
            ],
            tickfont=dict(size=14),
            len=0.5,
            lenmode="fraction",
            thickness=20,
            thicknessmode="pixels",
            titlefont=dict(size=14),
        ),
        dict(
            orientation="h",
            x=0.5,
            y=0.0,
            xanchor="center",
            yanchor="bottom",
            title="Fraction of mitigation in 2030 non-domestic",
            tickvals=[-1, -0.5, 0, 0.5, 1],
            ticktext=[
                "100%<br>More allocated than cost-optimal",
                "50%",
                "0%",
                "50%",
                "100%<br>Less allocated than cost-optimal",
            ],
            tickfont=dict(size=14),
            len=0.5,
            lenmode="fraction",
            thickness=20,
            thicknessmode="pixels",
            titlefont=dict(size=14),
        ),
    ][i]

    # Grey out nans
    if i < 2:
        wh = np.where(np.isnan(dat))[0]
        fig.add_trace(
            go.Choropleth(
                locations=list(np.array(dath.Region)[wh]) + ["GRL"],
                z=[-0.5] * (len(dat[wh]) + 1),
                locationmode="ISO-3",
                colorscale="Greys",
                zmax=1,
                zmin=-1,
                hovertemplate="%{text}",
                name="",
                marker_line_color="black",
                marker_line_width=0,
                showscale=False,
            ),
            [1, 1, 2, 2][i],
            [1, 2, 1, 2][i],
        )
    else:
        fig.add_trace(
            go.Choropleth(
                locations=list(np.array(data_def.Region)) + ["GRL"],
                z=[-0.5] * (len(np.array(data_def.Region)) + 1),
                locationmode="ISO-3",
                colorscale="Greys",
                zmax=1,
                zmin=-1,
                hovertemplate="%{text}",
                name="",
                marker_line_color="black",
                marker_line_width=0,
                showscale=False,
            ),
            [1, 1, 2, 2][i],
            [1, 2, 1, 2][i],
        )

    fig.add_trace(
        go.Choropleth(
            locations=np.array((dath).Region),
            z=dat,
            locationmode="ISO-3",
            colorscale=scale,
            zmax=1,
            zmin=-1,
            text=[
                str(r) + ": " + str(np.round(100 * dat[r_i], 1)) + " %"
                for r_i, r in enumerate(np.array((dath).Region))
            ],
            hovertemplate="%{text}",
            name="",
            colorbar=dc,
            marker_line_color="black",
            marker_line_width=0.75,
            showscale=True,
        ),
        [1, 1, 2, 2][i],
        [1, 2, 1, 2][i],
    )

    # EU
    fig.add_trace(
        go.Choropleth(
            locations=group_eu,
            z=[dat[np.where(np.array((dath).Region) == "EU")[0][0]]] * len(group_eu),
            locationmode="ISO-3",
            colorscale=scale,
            zmax=1,
            zmin=-1,
            # text = [str(r)+": "+str(np.round(100*dat[r_i], 1))+" %" for r_i, r in enumerate(group_eu)],
            hovertemplate="%{text}",
            name="",
            marker_line_color="black",
            marker_line_width=0.75,
            showscale=False,
        ),
        [1, 1, 2, 2][i],
        [1, 2, 1, 2][i],
    )

fs = 20
fig["layout"].update(
    annotations=[
        dict(
            x=0.03,
            y=1.0,
            text="<b>(a) Comparison with NDCs</b>",
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(color="black", size=fs),
        ),
        dict(
            x=0.53,
            y=1.0,
            text="<b>(b) Comparison with NDCs",
            xref="paper",
            yref="paper",
            xanchor="left",
            showarrow=False,
            font=dict(color="black", size=fs),
        ),
        dict(
            x=0.03,
            y=0.48,
            text="<b>(c) Comparison with cost-optimal pathways",
            xref="paper",
            yref="paper",
            xanchor="left",
            showarrow=False,
            font=dict(color="black", size=fs),
        ),
        dict(
            x=0.53,
            y=0.48,
            text="<b>(d) Comparison with cost-optimal pathways",
            xref="paper",
            yref="paper",
            xanchor="left",
            showarrow=False,
            font=dict(color="black", size=fs),
        ),
        dict(
            x=0.14,
            y=1.055,
            text="<i>Default: gap is {} Gt CO<sub>2</sub>e in 2030".format(
                np.round(GAP_DEFAULT / 1e3, 1), int(np.round(gap_default_perc * 100, 0))
            ),
            xref="paper",
            yref="paper",
            xanchor="left",
            showarrow=False,
            font=dict(color="black", size=fs),
        ),
        dict(
            x=0.62,
            y=1.055,
            text="<i>Maximum: gap is {} Gt CO<sub>2</sub>e in 2030".format(
                np.round(GAP_MAX / 1e3, 1), int(np.round(gap_max_perc * 100, 0))
            ),
            xref="paper",
            yref="paper",
            xanchor="left",
            showarrow=False,
            font=dict(color="black", size=fs),
        ),
    ]
)

fig.update_geos(
    visible=False,
    showlakes=False,
    lakecolor="rgb(255, 255, 255)",
    projection_type="natural earth",
    resolution=110,
    showcoastlines=False,
)
fig.update_traces(colorbar=dict(title=dict(side="bottom")))
fig.update_geos(landcolor="grey")
args = dict(xref="paper", yref="paper", type="line")
fig.add_shape(x0=0.01, y0=1.02, x1=0.495, y1=1.02, **args, line=dict(color="black", width=2))
fig.add_shape(x0=0.01, y0=1.00, x1=0.01, y1=1.02, **args, line=dict(color="black", width=2))
fig.add_shape(x0=0.495, y0=1.00, x1=0.495, y1=1.02, **args, line=dict(color="black", width=2))
fig.add_shape(x0=0.505, y0=1.02, x1=0.99, y1=1.02, **args, line=dict(color="black", width=2))
fig.add_shape(x0=0.505, y0=1.00, x1=0.505, y1=1.02, **args, line=dict(color="black", width=2))
fig.add_shape(x0=0.99, y0=1.00, x1=0.99, y1=1.02, **args, line=dict(color="black", width=2))
fig.update_layout(height=1000, width=1500)
fig.update_layout(
    plot_bgcolor="white",  # Background of the plot area
    paper_bgcolor="white",  # Background of the entire figure
)
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=50)  # l: left, r: right, t: top, b: bottom
)
fig.write_image("K:/Code/effort-sharing/Figures/Paper_FairShares/V3/Figure_3.png", scale=5)
fig.write_image("K:/Code/effort-sharing/Figures/Paper_FairShares/V3/SVG/Figure_3.svg", scale=5)
fig.show()